
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
 
# Also import the libraries shown below. You will use the urllib.request module to open URLs. And you’ll use  BeautifulSoup to extract data from html files. The Beautiful Soup library's name is bs4 which stands for BeautifulSoup, version 4.

# from urllib.request import urlopen
# from bs4 import BeautifulSoup

# After importing necessary modules, you should specify the URL containing the dataset and pass it to urlopen() to get the html of the page.

# url = "http://www.hubertiming.com/results/2017GPTR10K"
# html = urlopen(url)



In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
import bs4
# from bs4 import BeautifulSoup #for some reason this line is not working
url = "http://www.hubertiming.com/results/2017GPTR10K"
html = urlopen(url)

soup = bs4.BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

# Get the title
title = soup.title
# print(title)

# print out the text
text = soup.get_text()
# print(soup.text)

all_links = soup.find_all('a')
for link in all_links:
    print(link.get("href"))

# print out the first 10 rows for sanity check
rows = soup.find_all('tr')
# print(rows[:10])

for row in rows:
    row_td = row.find_all('td')
# print(row_td)
type(row_td)

bs4.element.ResultSet

# To remove html tags using Beautiful Soup, pass the string of interest into BeautifulSoup() 
# and use the get_text() method to extract the text without html tags

str_cells = str(row_td)
cleantext = bs4.BeautifulSoup(str_cells, 'lxml').get_text()
# print(cleantext)

#To gain a better understanding of the problem, try doing the same extraction with regular expressions.
# Be sure to import the re (regular expressions) module in your python environment. The code below shows
# how to build a regular expression that finds all the characters inside <td> html tags and replace them with
# an empty string for each table row

# First, the code compiles a regular expression by passing the regex to re.compile(). Compilation is not required,
# but many python programming resources recommend compilation of regex to improve performance. This is esp the case 
# if the same regex is to be used over many lines of input text

import re
list_rows= []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    clean = re.compile('<.*?>')
    clean2 = (re.sub(clean, '', str_cells))
    list_rows.append(clean2)
print(clean2)
type(clean2)

# The next step is to convert the list into a pandas dataframe and get a quick view of the first 10 rows of data.
# Pandas is a wonderful tool for manipulating python data as tables. It's almost like having a database server or google sheets
# within your python envir. We will use a couple of features in ths lab (and throughout the DataEng course), but I encourage you to
# explore it much further, see the pandas wiki page for a start
df = pd.DataFrame(list_rows)
df.head(10)

# Data Transformation
# Our topic this week is "Data Gathering", and in a sense, you are done with the gathering. But to make things interesting you should continue 
# on and use the data that you gathered. Next we need to transform the data into something more meaningful. 

# The dataframe is not in the format we want. To clean it up, split the "0" column into multiple columns at the comma position. 
# Use the str.split() method

df1 = df[0].str.split(',', expand=True)
df1.head(10)

# This looks much better, but there is still work to do. The dataframe has unwanted square brackets surrounding each row. Use the 
# strip() method to remove the opening square bracket on column "0"

df1[0] = df1[0].str.strip('[')
df1.head(10)

# The table is missing table headers. So go back to BeautifulSoup and use its find_all() method to get the table headers
col_labels = soup.find_all('th')
all_header = []
col_str = str(col_labels)
cleantext2 = bs4.BeautifulSoup(col_str, "lxml").get_text()
all_header.append(cleantext2)
print(all_header)

# Next convert the table headers to a new pandas dataframe

df2 = pd.DataFrame(all_header)
df2.head()

# Again, split column "0" into multiple columns at the comma position for all rows
df3 = df2[0].str.split(',', expand=True)
df3.head()

# Next, concatenate the two dataframes into one using the concat() method
frames = [df3, df1]

df4 = pd.concat(frames)
df4.head(10)

# Next, re-configure the data frame so that the first row is the table header
df5 = df4.rename(columns=df4.iloc[0])
df5.head()

# Behold all of the progress that you have made! At this point, the table is almost properly formatted.
# For analysis, start by getting an overview of the data as shown below
df5.info()
df5.shape

# The table has 583 rows and 8 columns. One statement that you will hear me say over and over throughout
# the course is, "if you did not validate the data then it's probably wrong", and this data is no exception
# How do we know? Because it has 583 rows, but the various columns have varying numbers of "non-null object"
# values. So transform it again to drop all rows with any missing values.

df6 = df5.dropna(axis=0, how='any')
df6.info()
df6.shape

# wasn't that easy? When I say "pandas is useful", this is 1 example of what I mean. YOu can do the same thing
# of thing within a relational database, but by keeping all of the data in memory within our python program we can
# accomplish this type of transformation task much more quickly and cleanly

# In a real data engineering project probably we would validate the data thoroughly by devising predicates for every 
# aspect of the data. But to keep the focus on data gathering we will instead move forward with the data we have.

# Notice how the table header is replicated as the first row in df5 and df6. Drop this redundant row like this:

df7 = df6.drop(df6.index[0])
df7.head()

# Clean up the headers a bit more by renaming the '[Place' and 'Team]' columns. Python is picky about whitespace.
# Make sure you include a space after the quotation mark in 'Team]'.
df7.rename(columns={'[Place': 'Place'}, inplace=True)
df7.rename(columns={' Team]': 'Team'}, inplace=True)
df7.head()

# The final data cleaning step involves removing the closing bracket for cells in the "team" column
df7['Team'] = df7['Team'].str.strip(']')
df7.head()

# The dataframe is now in the desired format. One thing you could do at this point is to further transform the data
# by eliminating redundancies and the unneeded information. For example, some of the columns appear to be redundant 
# and potentially could be dropped.

# Next move on to the Data Science part, including computation of summary statistics and plotting of results.

# DATA ANALYSIS AND VISUALIZATION

# The first question to answer is, 'What was the average finish time (in minutes) for the runners?' unfortunately, the "chip Time"
# field gives the finish time in hh:mm:ss, so you need to transform that column into just minutes. One way to do this is convert the column
# to a list first for manipulation

time_list = df7[' Chip Time'].tolist()

# You can use a for-loop to convert 'Chip Time' to minutes

time_mins = []
print(time_list)
for i in time_list:
    m, s = i.split(':')
    math = (int(m) * 60 + int(s))/60
    time_mins.append(math)
print(time_mins)


mailto:timing@hubertiming.com
https://www.hubertiming.com
/results/2017GPTR
/results/team/2017GPTR
/results/team/2017GPTR10K
/results/summary/2017GPTR10K
None
#tabs-1
https://www.hubertiming.com/
https://facebook.com/hubertiming/
None
[577, 443, 

                    LIBBY B MITCHELL

                , F, HILLSBORO, 1:41:18, 1:42:10, ]
['[Place, Bib, Name, Gender, City, Chip Time, Gun Time, Team]']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 0 to 581
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   [Place      583 non-null    object
 1    Bib        581 non-null    object
 2    Name       578 non-null    object
 3    Gender     578 non-null    object
 4    City       578 non-null    object
 5    Chip Time  578 non-null    object
 6    Gun Time   578 non-null    object
 7    Team]      578 non-null    object
dtypes: object(8)
memory usage: 41.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 578

ValueError: too many values to unpack (expected 2)